# Akka.Streams and Backoff
Part of the raison d'être for Akka.Streams in the first place is to faciliate support for [Reactive Streams](https://www.reactive-streams.org/) - the ability to ensure that in asynchronous producer-consumer systems a faster upstream producer can't overwhelm a slower downstream consumer.

Let's consider an example.

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample2");
IMaterializer materializer = actorSystem.Materializer();

// Fast upstream producer - instantly produces 10,000 requests
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));

// Group into batches of 100
Source<IEnumerable<int>,NotUsed> batchingSource = source.Via(Flow.Create<int>().Grouped(100)); 

// output
var (task, sink) = Sink.ForEach<int>(c => Console.WriteLine(c.ToString()))
    .PreMaterialize(materializer);
//.WithAttributes(new Attributes(new ActorAttributes.Dispatcher("akka.actor.synchronized-dispatcher")));

// simulate a slower downstream consumer - can only process 10 events per second
var slowSink = Flow.Create<IEnumerable<int>>().Delay(TimeSpan.FromMilliseconds(100))
    .Select(x => x.Sum()) // sum each group
    .To(sink);

batchingSource.RunWith(slowSink, materializer);

await task;


Installed Packages Akka.Streams, 1.4.24